In [4]:
import gzip
from gensim.models import Word2Vec, LdaMulticore
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.corpora import Dictionary

import numpy as np
import pandas as pd
import pyLDAvis
from pyLDAvis import gensim

## Dane

In [33]:
ls -lh data

 Volume in drive C is Windows
 Volume Serial Number is 1A25-A5F3

 Directory of C:\Users\tjb1dl


 Directory of C:\Users\tjb1dl\data

11/15/2019  04:26 PM    <DIR>          .
11/15/2019  04:26 PM    <DIR>          ..
11/15/2019  03:18 PM       528,766,166 models.zip
09/03/2019  10:30 AM       240,342,787 reviews_data.txt
               2 File(s)    769,108,953 bytes
               2 Dir(s)  163,678,318,592 bytes free


File Not Found


In [80]:
def preprocessing(sentence):
    return [word for word in simple_preprocess(sentence) 
            if word not in STOPWORDS]
    
def read_sentence(filename):
    f = open(filename, 'rb')
    for line in f:
        yield preprocessing(line)


In [81]:
sentences = list(read_sentence('data/reviews_data.txt'))

## Model

In [82]:
model = Word2Vec(sentences, size=100, min_count=2, window=4)

## Reprezentacja wektorowa

In [97]:
model.wv.most_similar('denver')

[('newcastle', 0.7528369426727295),
 ('nc', 0.7500677108764648),
 ('pch', 0.7409628629684448),
 ('qm', 0.7404193878173828),
 ('detroit', 0.7329400777816772),
 ('birmingham', 0.7256693243980408),
 ('devon', 0.7242261171340942),
 ('alaska', 0.7238695621490479),
 ('kl', 0.7207643389701843),
 ('tx', 0.7187349796295166)]

## Topic modeling - LDA


In [98]:
sentences_light = np.random.permutation(sentences)

In [106]:
sentences_light = sentences_light[:1000]

In [107]:
%time dictionary = Dictionary(sentences_light)

Wall time: 122 ms


In [109]:
len(dictionary)

8888

In [111]:
bow_corpus = [dictionary.doc2bow(sent) for sent in sentences_light]

In [113]:
dictionary.doc2bow(['car'])

[(78, 1)]

In [115]:
dictionary.token2id['car']

78

In [116]:
lda_model = LdaMulticore(bow_corpus, id2word=dictionary, num_topics=100,passes=20, workers=8)

In [120]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx,topic))

Topic: 0 
Words: 0.026*"hotel" + 0.023*"great" + 0.015*"place" + 0.015*"room" + 0.014*"square" + 0.010*"union" + 0.009*"helpful" + 0.009*"extremely" + 0.009*"staff" + 0.008*"size"
Topic: 1 
Words: 0.018*"food" + 0.018*"london" + 0.017*"stay" + 0.016*"quot" + 0.016*"quiet" + 0.012*"yes" + 0.012*"goldilocks" + 0.012*"sightseeing" + 0.012*"pub" + 0.011*"room"
Topic: 2 
Words: 0.038*"room" + 0.019*"hotel" + 0.010*"nice" + 0.010*"stay" + 0.009*"place" + 0.008*"good" + 0.007*"small" + 0.007*"rooms" + 0.006*"car" + 0.006*"water"
Topic: 3 
Words: 0.024*"hotel" + 0.020*"room" + 0.016*"location" + 0.016*"clean" + 0.016*"food" + 0.012*"time" + 0.012*"staff" + 0.012*"good" + 0.012*"min" + 0.008*"want"
Topic: 4 
Words: 0.029*"hotel" + 0.018*"good" + 0.012*"room" + 0.011*"rooms" + 0.011*"buffet" + 0.010*"location" + 0.008*"reasonable" + 0.008*"fed" + 0.008*"knew" + 0.008*"spectacular"
Topic: 5 
Words: 0.022*"hotel" + 0.020*"room" + 0.018*"good" + 0.012*"stay" + 0.011*"location" + 0.009*"stayed" + 0.

## Wizualizacja tematów

In [121]:
%time lda_vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.display(lda_vis)

C:\Users\tjb1dl\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


Wall time: 2min 12s


## Podobne słowa

In [147]:
model = Word2Vec(sentences, size=50, window=4, min_count = 1)

In [153]:
model.wv.most_similar('10', topn=3)

[('8', 0.3663967251777649),
 ('7', 0.35577917098999023),
 ('12', 0.35414618253707886)]

## Skutek uboczny - nauka matematyki - typ unspupervised ale możemy dotrenować używając gotowych wyników i nauki nadzorowanej

In [149]:
model.wv.most_similar(positive = ['10', '8'], negative=['5'], topn=3)

[('12', 0.3942561745643616),
 ('11', 0.36022740602493286),
 ('13', 0.29416924715042114)]